<div style="clear: both;">
<div>
      <h2><b>EXPORT</b>4PHOBS</h2>
     <img src="https://scontent.flju1-1.fna.fbcdn.net/v/t39.30808-1/538848259_1373858921407395_3734947029332493437_n.jpg?stp=c221.221.1034.1034a_dst-jpg_s480x480_tt6&_nc_cat=110&ccb=1-7&_nc_sid=2d3e12&_nc_ohc=nLyDiOB5EM8Q7kNvwEw-9Xf&_nc_oc=Adm0RsJmASURcGiXVvWUyJRIrypsmZsQEM5WYIv2uG8ZFEp5f1wteeqopUsLj6bPgFk&_nc_zt=24&_nc_ht=scontent.flju1-1.fna&_nc_gid=J2ABA-OjzFAu8lGZyNEJ3g&oh=00_AffBA_KZ8YxYLhFZ2GHdTmTHnUp2bG6tZWV73qMxnj66Qg&oe=68FAB123" width="180" alt="">
  </div>
  </div>
  <div>
    <p>Generator za BAR cenike</p>
  </div>
</div>



In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import base64
import urllib.parse
import time

output = widgets.Output()

# --- Function to prepare PHOBS dataframe ---
def prepare_phobs_csv(df, hotel_id, los_code):
    if 'BAR' not in df.columns:
        df['BAR'] = 120  # default BAR value

    df['BAR'] = df['BAR'].apply(lambda x: f"BAR{x}")
    df['Hotel_ID'] = hotel_id
    df['nicla'] = 0
    df['Yield'] = f"YIELD{los_code}"

    # Ensure 'Datum' exists
    if 'Datum' not in df.columns:
        df['Datum'] = pd.Timestamp.today().strftime('%Y-%m-%d')

    df = df[['Hotel_ID', 'Datum', 'nicla', 'BAR', 'Yield']]
    return df

# --- Function to create a download button ---
def create_download_button(df, filename, description):
    button = widgets.Button(
        description=description,
        button_style='info',
        layout=widgets.Layout(width="33%")
    )

    def on_click(b):
        with output:
            output.clear_output()
            display(HTML(f"<b>{description}</b> CSV download started..."))
            csv_bytes = df.to_csv(index=False, header=False).encode()
            b64 = base64.b64encode(csv_bytes).decode()

            # JS with fallback link
            js = f"""
            <a download="{filename}.csv" href="data:text/csv;base64,{b64}" id="download_link">Download</a>
            <script>
            const link = document.getElementById('download_link');
            link.click();
            setTimeout(() => link.remove(), 5000);
            </script>
            """
            display(HTML(js))

    button.on_click(on_click)
    return button

# --- Load master PHOBS sheet ---
gsheet_id = "15HJ7wxyUmo-gcl5_y1M9gl4Ti-JSsYEJZCjoI76s-Xk"
master_sheet_url = f"https://docs.google.com/spreadsheets/d/{gsheet_id}/gviz/tq?tqx=out:csv&sheet=PHOBS"
master_df = pd.read_csv(master_sheet_url)

# Generate hotels list
hotels = [(row['Hotel_Name'], row['Hotel_ID'], row['YIELD_Code']) for _, row in master_df.iterrows()]

buttons = []
with output:
    display(HTML("<b>Loading hotel sheets...</b>"))

# --- Load each hotel's sheet and create buttons ---
for hotel_name, hotel_id, los_code in hotels:
    try:
        sheet_name_encoded = urllib.parse.quote(hotel_name)
        sheet_url = f"https://docs.google.com/spreadsheets/d/{gsheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name_encoded}"
        df = pd.read_csv(sheet_url)
        df = prepare_phobs_csv(df, hotel_id, los_code)
        button = create_download_button(df, f"{hotel_name}-PHOBS", hotel_name.replace("_", " "))
        buttons.append(button)
    except Exception as e:
        with output:
            display(HTML(f"<b style='color:red'>Failed to load sheet for {hotel_name}:</b> {e}"))

# --- Layout buttons ---
rows = [widgets.HBox(buttons[i:i+3]) for i in range(0, len(buttons), 3)]
vbox = widgets.VBox(rows)

output.clear_output()
display(vbox, output)
